# Mapping Evictions from 2010 - 2019

This is one of three displacement indicators that is mapping evictions. This notebook uses folium to create a static map of evictions in Westlake, Jefferson Park, Lincoln Heights, Boyle Heights, Los Feliz, and Fairfax. 

I cleaned the code, used a loop to label neighborhoods and defined a function to filter the data.

In [1]:
import pandas as pd

import geopandas as gpd


Import eviction data, this data came from Eviction Lab. The data includes eviction data for all of the United States from 2000-2020. 

In [2]:
df= pd.read_csv('eviction_data.csv',
 dtype=
    {
        'fips':'str',

    }
)

Filtered down eviction data to include only data from Los Angeles County.

In [3]:
df_la=df[df['county']=='Los Angeles County']

Filtered eviction data for years 2010-2019.

In [4]:
df_la=df_la[(df_la['year']>=2010) & (df_la['year']<=2019)]

Imported data that allows us to map fips onto our desired neighborhoods. This data came from censusreporter.org. Census Reporter is an independent project to make data from the American Community Survey (ACS) easier to use. 

In [5]:
fips_to_neighborhood = pd.read_csv('tiger2020_12895e183b0c022d5a527c612ce72865_block_assignments.csv', dtype={'geoid':'str'})
fips_to_neighborhood['geoid'] = fips_to_neighborhood['geoid'].str[1:-4]

Defined a function to filter the data based on neighborhood for each neighborhood we are interested in, to get the fips that are associated with our target neighborhoods.

In [6]:
def get_fips(neighborhood_name):
    return list(set(fips_to_neighborhood[fips_to_neighborhood['name']== neighborhood_name]['geoid']))

In [7]:
westlake_fips = get_fips('Westlake') 
jefferson_park_fips = get_fips('Jefferson Park') 
lincoln_heights_fips = get_fips('Lincoln Heights') 
boyle_heights_fips = get_fips('Boyle Heights') 
los_feliz_fips = get_fips('Los Feliz') 
fairfax_fips = get_fips('Fairfax') 


target_neighborhood_fips = westlake_fips + jefferson_park_fips + lincoln_heights_fips + boyle_heights_fips + los_feliz_fips+ fairfax_fips

Once I had the fips for our target neighborhoods, I then filtered the eviction to focus in on the neighborhoods specified.

In [8]:
df_target_neighborhoods=df_la[df_la['fips'].isin(target_neighborhood_fips) ]

Added up all the evictions for each year. 

In [9]:
df_target_neighborhoods = df_target_neighborhoods.groupby(df_target_neighborhoods['fips']).aggregate({'filings': 'sum'})

Imported census tract data so I could map fips to polygons.

In [16]:
tracts=gpd.read_file('Census_Tracts_2020.geojson')

Created a fips column so that it will be easier to merge our eviction data with our polygon data. I had to remove the leading 0 because our other data source labeled fips without it.

In [11]:
tracts['fips'] ='6' + '037' + tracts['CT20']

Merged the eviction and polygon data 

In [12]:
tracts_df_target_neighborhoods=tracts.merge(df_target_neighborhoods,on="fips", how="right")

In [13]:
import folium

Mapped it using a high contrast color scheme relative to the base map

In [14]:
# fill n/a filings otherwise map colors polygons black
tracts_df_target_neighborhoods = tracts_df_target_neighborhoods.fillna(0)

m = folium.Map(location=[34.2,-118.2], 
               zoom_start = 9,
               tiles='CartoDB positron', 
               attribution='CartoDB')


folium.Choropleth(
                  geo_data=tracts_df_target_neighborhoods, 
                  data=tracts_df_target_neighborhoods,          
                  key_on='feature.properties.fips', 
                  columns=['fips', 'filings'], 
                  fill_color='RdYlBu',
                  line_weight=0.7, 
                  fill_opacity=.75,
                  bins=list(range(0, 400, 25)),
                  line_opacity=0.9, 
                  legend_name='Total Filings)').add_to(m)    

In [15]:
m

This map shows the total eviction filings (whose information was available) in Westlake, Fairfax, Lincoln Heights, Boyle Heights, Los Feliz, and Jefferson Park. 